# LangChain & Prompt Engineering을 이용하여 감성 분석과 맥락을 통해 근거 추론하기

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
model = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

In [5]:
conversation = [{
  "content": "안녕하세요! 처음 뵙겠습니다.",
  "name": "지한"
},
{
  "content": "안녕하세요! 처음 만나서 반갑습니다. 커피 한 잔 어떠세요?",
  "name": "수아"
},
{
  "content": "안녕하세요! 저도 처음 만나서 반갑습니다. 네, 커피 한 잔 같이 마시면 좋겠네요. 어떤 커피가 좋으세요?",
  "name": "지한"
},
{
  "content": "저는 보통 아메리카노나 카페 라떼를 좋아해요. 하지만 여러분의 취향에 맞춰서 골라도 괜찮아요. 어떤 커피를 좋아하시나요?",
  "name": "수아"
},
{
  "content": "아메리카노와 카페 라떼 중에 한 가지를 고르는 것도 좋을 것 같아요. 저는 보통 아메리카노를 주로 마시는 편이에요. 그래도 제게는 별로 상관없어요. 함께 고민하면서 결정해도 될까요?",
  "name": "지한"
},
{
  "content": "네, 함께 결정하면서 고민해도 되요! 그러면 저는 카페 라떼로 주문하겠습니다. 그래도 제게는 상관없어요. 커피 주문하러 가볼까요?",
  "name": "수아"
},
{
  "content": "네, 그렇게 해요. 카페 라떼로 주문할게요. 커피 주문하러 가볼까요? 커피 맛을 즐기면서 이야기도 나눌 수 있어서 좋네요.",
  "name": "지한"
},
{
  "content": "그래요, 커피 주문하러 가요. 커피 맛을 즐기면서 이야기도 나누는 건 정말 좋은 방법이에요. 커피가 나오면서 우리가 좀 더 편하게 대화할 수 있겠죠. 여기 커피 맛집인 것 같아요. 주문하러 가볼까요?",
  "name": "수아"
}]


In [6]:
conv_text = ""

for turn in conversation:
    conv_text += f"{turn['name']}: {turn['content']}\n"

In [7]:
print(conv_text)

지한: 안녕하세요! 처음 뵙겠습니다.
수아: 안녕하세요! 처음 만나서 반갑습니다. 커피 한 잔 어떠세요?
지한: 안녕하세요! 저도 처음 만나서 반갑습니다. 네, 커피 한 잔 같이 마시면 좋겠네요. 어떤 커피가 좋으세요?
수아: 저는 보통 아메리카노나 카페 라떼를 좋아해요. 하지만 여러분의 취향에 맞춰서 골라도 괜찮아요. 어떤 커피를 좋아하시나요?
지한: 아메리카노와 카페 라떼 중에 한 가지를 고르는 것도 좋을 것 같아요. 저는 보통 아메리카노를 주로 마시는 편이에요. 그래도 제게는 별로 상관없어요. 함께 고민하면서 결정해도 될까요?
수아: 네, 함께 결정하면서 고민해도 되요! 그러면 저는 카페 라떼로 주문하겠습니다. 그래도 제게는 상관없어요. 커피 주문하러 가볼까요?
지한: 네, 그렇게 해요. 카페 라떼로 주문할게요. 커피 주문하러 가볼까요? 커피 맛을 즐기면서 이야기도 나눌 수 있어서 좋네요.
수아: 그래요, 커피 주문하러 가요. 커피 맛을 즐기면서 이야기도 나누는 건 정말 좋은 방법이에요. 커피가 나오면서 우리가 좀 더 편하게 대화할 수 있겠죠. 여기 커피 맛집인 것 같아요. 주문하러 가볼까요?



In [8]:
class Sentiment(BaseModel):
    thought: str = Field(description="Sentiment를 판단하기 위한 근거")
    sentiment: str = Field(description="Sentiment: Positive | Negative")

In [9]:
parser = JsonOutputParser(pydantic_object=Sentiment)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"thought": {"title": "Thought", "description": "Sentiment를 판단하기 위한 근거", "type": "string"}, "sentiment": {"title": "Sentiment", "description": "Sentiment: Positive | Negative", "type": "string"}}, "required": ["thought", "sentiment"]}\n```'

In [10]:
human_prompt_template = HumanMessagePromptTemplate.from_template(
                            "다음 대화의 내용에 대해 감성분석해줘.\n{input}\n{format_instructions}")

prompt = ChatPromptTemplate.from_messages(
    [
        human_prompt_template
    ])
prompt = prompt.partial(format_instructions=format_instructions)

In [11]:
sentiment_analysis_chain = prompt | model | parser

In [12]:
sentiment_analysis_chain.invoke({"input": conv_text})

{'thought': '대화 내용은 서로 반갑게 인사하고 커피를 함께 마시기로 하는 긍정적인 분위기를 담고 있습니다. 서로의 커피 취향을 공유하며, 함께 결정하는 과정에서 상호 존중과 배려가 느껴집니다.',
 'sentiment': 'Positive'}